## @yullano90 | M ● Manipulação de Data e Hora

In [ ]:
# Avançando, vamos utilizar a função "DateTime.LocalNow()" para saber a data atualizada do S.O.

-------------------------------------------------------------------------------------------------

    //Função para saber a data atualizada do S.O:
    FuncCheckAniversario = (data)=>
    let //Especificação de variáveis:
        DataAtual = Date.From(DateTime.LocalNow()),
        Dia = Date.Day(data),
        Mes = Date.Month(data),
        Ano = Date.Year(data),
        DiaAtual = Date.Day(DataAtual),
        MesAtual = Date.Month(DataAtual)
    in
        Dia = DiaAtual and Mes = MesAtual,

-------------------------------------------------------------------------------------------------

In [ ]:
# Em seguida, vamos preecher uma coluna informando se o aluno está aniversariando ou não.
-------------------------------------------------------------------------------------------------

    //Adicionando coluna de Aniversariante:
    AddC_Aniversario =
        Table.AddColumn(
            AddColumnProporcao,
            "Aniversariante",
            each FuncCheckAniversario(
                [Dt_Nascimento]
            )
        )    

-------------------------------------------------------------------------------------------------


In [ ]:
# O resultado esperado é uma código semelhante a este:

-------------------------------------------------------------------------------------------------
let
    //Com a adição da coluna "Idade" o texto do Json aumenta ligeiramente:
    Fonte =
        Table.FromRows(Json.Document(Binary.Decompress(Binary.FromText(
        "TY5LCsMwDETv4rVBkj+1vcy2EAjNMvgGpV31/p2xU9KFHljoeeY43PL8vN7mvL
        thCsaUSMDjDgRVLkIUrWKtqev+tAL2DVMxeVj01x2I07IgphI02GXF06g+0+dZI
        vZtfDO9KhqRVvLlsRET2bVSLflnWZ4dVTQhTf8s3jBx2LMSsC0AX5oZBCW53r8=", 
        BinaryEncoding.Base64), Compression.Deflate)), 
        let _t = ((type nullable text) meta [Serialized.Text = true]) 
        in type table [
                        Nome = _t, 
                        Nota1 = _t, 
                        Nota2 = _t, 
                        Frequencia = _t, 
                        Idade = _t, 
                        UF = _t, 
                        Investimento = _t, 
                        Dt_Nascimento = _t
            ]
        ),

    //Importante normalizar tipos de dados antes de realizar operações matemáticas:
    AlterType =
        Table.TransformColumnTypes(
            Fonte,{
                    {"Nome", type text},
                    {"Nota1",  type number},
                    {"Nota2", type number},
                    {"Frequencia",  type number},
                    {"Idade", type number},
                    {"Investimento", type number},
                    {"Dt_Nascimento", type date}
                }
            ),

    //Criação de uma simples função como exemplo:
    FuncMeanNotes =
        (Nota1, Nota2) => (Nota1 + Nota2) / 2,

    //Coluna adicionando a função "FuncMeanNotes" com o "EACH":
    AddColumnMeanNotes =
        Table.AddColumn(
            AlterType, "Média", each FuncMeanNotes([Nota1],[Nota2])
        ),

    //Coluna de status do aluno, retornando um valor booleano:
    AddAproveColumn =
        Table.AddColumn(
            AddColumnMeanNotes, "Aprovação", each [Frequencia] >=70 and [Média] >= 7
        ),

    //Cálculo do Status para votação:
    FuncStatusVotacao =
        (idade) => if idade < 16 then "Não vota"
        else if idade < 18 then "Facultativo"
        else if idade < 70  then "Obrigatório"
        else "Facultativo",

    //Adicionar uma coluna invocando a função "FuncStatusVotacao":
    AddC_StatusVotacao = Table.AddColumn(
        AddAproveColumn, "Status_Votação", each FuncStatusVotacao([Idade])
        ),

    //Utilizando a função list.sum() passaremos a coluna a ser somada:
    FuncSomaInvestimento = List.Sum(AddC_StatusVotacao[Investimento]),

    //Função que faz contagem de linhas, que será o denominador do %:
    AddColumnProporcao =
        Table.AddColumn(
            AddC_StatusVotacao, "Proporção", each (
                [Investimento]/FuncSomaInvestimento * 100
                )
            ),

    //Função para saber a data atualizada do S.O:
    FuncCheckAniversario = (data)=>
    let //Especificação de variáveis:
        DataAtual = Date.From(DateTime.LocalNow()),
        Dia = Date.Day(data),
        Mes = Date.Month(data),
        Ano = Date.Year(data),
        DiaAtual = Date.Day(DataAtual),
        MesAtual = Date.Month(DataAtual)
    in
        Dia = DiaAtual and Mes = MesAtual,


    //Adicionando coluna de Aniversariante:
    AddC_Aniversario =
        Table.AddColumn(
            AddColumnProporcao,
            "Aniversariante",
            each FuncCheckAniversario(
                [Dt_Nascimento]
            )
        )                    

in
    AddC_Aniversario

-------------------------------------------------------------------------------------------------